In [ ]:
!pip install -Uq adapters
!pip install -q datasets
!pip install -q accelerate

In [ ]:
from transformers import DataCollatorWithPadding
import numpy as np
from datasets import load_dataset
from sklearn.model_selection import KFold

In [ ]:
from transformers import AutoConfig
from adapters import AutoAdapterModel

In [ ]:
#loading dataset from huggingface
from datasets import load_dataset

dataset = load_dataset("mediabiasgroup/mbib-base")
dataset.num_rows

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


{'cognitive_bias': 7092,
 'fake_news': 8542,
 'gender_bias': 17940,
 'hate_speech': 339010,
 'linguistic_bias': 401862,
 'political_bias': 17704,
 'racial_bias': 9788,
 'text_level_bias': 9018}

In [ ]:
#random shuffling (inter-mixing) of dataset
dataset1=dataset['text_level_bias'].shuffle(seed=42)


In [ ]:
# To replace nan values in dataset1 with ' '

dataset1 = dataset1.map(lambda x: {'text': ' ' if x['text'] is None else x['text']})

Map:   0%|          | 0/17704 [00:00<?, ? examples/s]

In [ ]:
# # replace nan value with empty string
# dataset1['text'] = dataset1['text'].fillna(' ')

In [ ]:
# generating 3 folds using KFold from sklearn
kf = KFold(n_splits=3)


In [ ]:
from transformers import AutoTokenizer

#tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-2022-154m")

In [ ]:
from transformers import AutoTokenizer

#tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-2022-154m")
def encode_batch(batch):
  """Encodes a batch of input data using the model tokenizer."""
  return tokenizer(batch["text"], padding=True,truncation=True, max_length=512)

# Encode the input data
dataset1 = dataset1.map(encode_batch, batched=True)
# The transformers model expects the target class column to be named "labels"
dataset1 = dataset1.rename_column("label", "labels")
# Transform to pytorch tensors and only output the required columns
dataset1.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/9018 [00:00<?, ? examples/s]

In [ ]:

id2label ={0:0,1:1}


In [ ]:
config = AutoConfig.from_pretrained("cardiffnlp/twitter-roberta-base-2022-154m")
model = AutoAdapterModel.from_pretrained(
    "cardiffnlp/twitter-roberta-base-2022-154m",
    config=config,
)

In [ ]:
from adapters.composition import Fuse

#Load the pre-trained adapters we want to fuse
model.load_adapter("SOUMYADEEPSAR/political_bias", with_head=False)
model.load_adapter("SOUMYADEEPSAR/gender_bias", with_head=False)
model.load_adapter("SOUMYADEEPSAR/racial_bias", with_head=False)
model.load_adapter("SOUMYADEEPSAR/cognitive_bias1", with_head=False)
model.load_adapter("SOUMYADEEPSAR/text_level_bias1", with_head=False)
# Add a fusion layer for all loaded adapters
adapter_setup = Fuse("political_bias", "gender_bias", "racial_bias","cognitive_bias1","text_level_bias1")
model.add_adapter_fusion(adapter_setup)

# Add a classification head for our target task
model.add_classification_head("cognitive_bias_fusion", num_labels=len(id2label))

In [ ]:
model.train_adapter_fusion(adapter_setup)

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
import numpy as np
from transformers import TrainingArguments, EvalPrediction,EarlyStoppingCallback
from adapters import AdapterTrainer

from sklearn.metrics import f1_score

training_args = TrainingArguments(
    learning_rate=5.5e-5,
    num_train_epochs=6,
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    #logging_steps=200,
    output_dir="./training_output",
    #overwrite_output_dir=True,
    load_best_model_at_end=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
    report_to="none"
)

def compute_metrics(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis=1)
  f1 = f1_score(p.label_ids, preds, average='macro')
  return {
      'macro_f1': f1,
  }



In [ ]:
for fold, (train_idx, test_idx) in enumerate(kf.split(dataset1)):
    train_idx1 = train_idx[:int((len(train_idx) )*0.8)]
    val_idx = train_idx[int((len(train_idx)) *0.8):]
    print(f"Fold {fold+1}")
    print(train_idx1)
    print(val_idx)
    print(test_idx)
    print(len(train_idx1))
    print(len(val_idx))
    print(len(test_idx))

Fold 1
[3006 3007 3008 ... 7812 7813 7814]
[7815 7816 7817 ... 9015 9016 9017]
[   0    1    2 ... 3003 3004 3005]
4809
1203
3006
Fold 2
[   0    1    2 ... 7812 7813 7814]
[7815 7816 7817 ... 9015 9016 9017]
[3006 3007 3008 ... 6009 6010 6011]
4809
1203
3006
Fold 3
[   0    1    2 ... 4806 4807 4808]
[4809 4810 4811 ... 6009 6010 6011]
[6012 6013 6014 ... 9015 9016 9017]
4809
1203
3006


In [ ]:
from adapters import AdapterTrainer

In [ ]:
f1_scores=[]
for fold, (train_idx, test_idx) in enumerate(kf.split(dataset1)):
    print(f"Fold {fold+1}")
    # 80 percent of train data from a fold was used for training and 20 percent for validation
    train_idx1 = train_idx[:int((len(train_idx) )*0.8)]
    val_idx = train_idx[int((len(train_idx)) *0.8):]
    print(f"Fold {fold+1}")
    # print(len(train_idx1))
    # print(len(val_idx))
    # print(len(test_idx))
    # Split dataset into training and validation based on the indices from KFold
    train_split = dataset1.select(train_idx1)
    val_split = dataset1.select(val_idx)
    test_split = dataset1.select(test_idx)
    print(len(train_split))
    print(len(val_split))
    print(len(test_split))

    # Load the model for classification and its configuration

    config = AutoConfig.from_pretrained("cardiffnlp/twitter-roberta-base-2022-154m")

    model = AutoAdapterModel.from_pretrained(
    "cardiffnlp/twitter-roberta-base-2022-154m",
    config=config,
    )

    from adapters.composition import Fuse

    #Load the pre-trained adapters we want to fuse
    model.load_adapter("SOUMYADEEPSAR/political_bias", with_head=False)
    model.load_adapter("SOUMYADEEPSAR/gender_bias", with_head=False)
    model.load_adapter("SOUMYADEEPSAR/racial_bias", with_head=False)
    model.load_adapter("SOUMYADEEPSAR/cognitive_bias1", with_head=False)
    model.load_adapter("SOUMYADEEPSAR/text_level_bias1", with_head=False)

    # Add a fusion layer for all loaded adapters
    adapter_setup = Fuse("political_bias", "gender_bias", "racial_bias","cognitive_bias1","text_level_bias1")
    model.add_adapter_fusion(adapter_setup)


    # Add a classification head for our target task
    model.add_classification_head("political_bias_fusion", num_labels=len(id2label))
    #activating adapter fusion layers for training and freezing all other parameters of model and single-task adapters
    model.train_adapter_fusion(adapter_setup)

    # Create a Trainer instance
    trainer = AdapterTrainer(
        model=model,
        args=training_args,
        train_dataset=train_split,
        eval_dataset=val_split,
        compute_metrics=compute_metrics,
        data_collator=data_collator,
        callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
    )

    # Train the model
    trainer.train()

    # Evaluate the model on validation data on kth fold
    metrics = trainer.evaluate()
    print(f"Metrics for valid fold {fold+1}: {metrics}")
    #Evaluate on test data
    outputs = trainer.predict(test_split)
    print(f"Metrics for test fold {fold+1}: {outputs.metrics}")
    f1_scores.append(outputs.metrics['test_macro_f1'])


Fold 1
Fold 1
4809
1203
3006
4809
1203
3006


Some weights of RobertaAdapterModel were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-2022-154m and are newly initialized: ['heads.default.3.bias', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/adapters/loading.py:165: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(weights_file, map_location="cpu")


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

...

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Epoch,Training Loss,Validation Loss,Macro F1
1,0.508900,0.497884,0.768910
2,0.480800,0.505773,0.776392
3,0.453400,0.496005,0.767150
4,0.455200,0.502396,0.774050
5,0.431200,0.499275,0.780536
6,0.419700,0.515328,0.776169


In [ ]:
# Find average of f1_scores from each folds in cross-validation

import numpy as np

# Assuming f1_scores is a list of f1 scores from each fold
average_f1 = np.mean(f1_scores)
print(f"Average F1 score: {average_f1}")

Average F1 score: 0.8771281148652367
